# <b> Project 1

<b>Introduction<br></b>
In this project, there will be three parts to analyze pandas. First part will be Grammar for pandas library. Second part will implement an algorithm to identify if the input is vaild DataFrame structure. Third part will be the cost analysis of DataFrame and ___ . 

The following BNF grammar definition of Pandas:


\begin{eqnarray*}
\textit{ mod}& ::= &\textbf{Module(stmt* body))} \\
                 &  |  &\textbf{Expression(expr body)} \\
\\
\textit{stmt}& ::= &\textbf{Assign(expr* targets, stmt value))} \\
\\
\\
\textit{expr}& ::= &\textbf{DataFrame(expr)} \\
                 &  |  &\textbf{Series(expr* value)} \\
                 &  |  &\textbf{ Num(object n)} \\
                 &  |  &\textbf{ Subscript(expr value, slice slice, expr_context ctx)} \\
                 &  |  &\textbf{ Str(string s)} \\
                 &  |  &\textbf{ Name(identifier id, expr_context ctx)} \\
                 &  |  &\textbf{ Dict(expr* keys,  expr* values)} \\
                 &  |  &\textbf{List(expr* elts, expr_context ctx)} \\
                 &  |  &\textbf{Tuple(expr* elts, expr_context ctx)} \\
                 &  |  &\textbf{Slice(expr* value)} \\
                 &  |  &\textbf{Call(expr func, expr* args, keyword* keywords)} \\
                 &  |  &\textbf{BinOp(expr left, operator op, expr right)} \\
                 \\
                 \\
\textit{operator}& ::= &\textbf{Add | Sub | Mult | MatMult | Div | Mod | Pow } \\
                   &  |  &\textbf{| LShift| RShift | BitOr | BitXor | BitAnd | FloorDiv} \\
\\
\textit{keyword arguments supplied to call:}\\
\textit{keyword}& ::= &\textbf{(identifier? arg, expr value) } \\
\end{eqnarray*}


<b>Part II</b>
<p>Below is an implementation of an algorithm that determine if the input statement can be transform or is already DataFrame or not
<br>

In [274]:
#This 
import ast 
import pandas as pd
class is_DataFrame(ast.NodeVisitor):
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        return all(results)

    def visit_Expr(self, node):
        return self.visit(node.value)
    
    def visit_Call(self, node):
        results = [self.visit(s) for s in node.keywords]
        return all(results)
    
    def visit_keyword(self, node):
        return self.visit(node.value)
    
    def visit_BinOp(self, node):
        return self.visit(node.left) and self.visit(node.right)

    def visit_List(self, node):
        return True
    
    def visit_Dict(self, node):
        return True
    
    def visit_Subscript(self, node):
        return True


test = ast.parse('pd.DataFrame(data = [1,2,3] + [9,2,3] )')
test1 = ast.parse('pd.DataFrame(data = 1)')
print(is_DataFrame().visit(test))
print(is_DataFrame().visit(test1))

Module(body=[Expr(value=Call(func=Attribute(value=Name(id='pd', ctx=Load()), attr='DataFrame', ctx=Load()), args=[], keywords=[keyword(arg='data', value=List(elts=[Num(n=1), Num(n=2), Num(n=3)], ctx=Load()))]))])
True
False


<b>Part III</b>
<p>a) Implementation of a static analysis algorithm that infers the type of a single DataFrame expression.

In [145]:
import ast 
import pandas as pd
'''class infer_type(ast.NodeVisitor):
    
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        if results == [int] or results == [bool]:
            return results[0]

    def visit_Expr(self, node):
        return self.visit(node.value)

    def visit_BoolOp(self, node):
        if set([self.visit(v) for v in node.values]) == {bool}:
            return bool

    def visit_BinOp(self, node):
        if type(node.op) in [ast.Add, ast.Sub, ast.Mult]:
            if self.visit(node.left) is int and\
               self.visit(node.right) is int:
                return int
    
    def visit_Compare(self, node):
        if type(node.ops[0]) in [ast.Eq]:
            if self.visit(node.left) is int and\
               self.visit(node.comparators[0]) is int:
                return bool
    
    def visit_Num(self, node):
        if type(node.n) is int:
            return int
    
    def visit_NameConstant(self, node):
        if node.value in [True, False]:
            return bool
    
print(infer_type().visit(ast.parse('5')))
print(infer_type().visit(ast.parse('5 + 5')))
print(infer_type().visit(ast.parse('True and False')))
print(infer_type().visit(ast.parse('False or 5')))
print(infer_type().visit(ast.parse('False + True')))
print(infer_type().visit(ast.parse('3 or 4 and 5')))
print(infer_type().visit(ast.parse('3 == 5')))'''

class infer_type(ast.NodeVisitor):
    
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        if results == [int] or results == [bool]:
            return results[0]

    def visit_Expr(self, node):
        return self.visit(node.value)

    def visit_Call(self, node):
        results = [self.visit(s) for s in node.keywords]
        if results == [int] or results == [bool]:
            return results[0]
    
    def visit_keyword(self, node):
        return self.visit(node.value)

    def visit_BinOp(self, node):
        if type(node.op) in [ast.Add, ast.Sub, ast.Mult]:
            if self.visit(node.left) is int and\
               self.visit(node.right) is int:
                return int
    
    def visit_Compare(self, node):
        if type(node.ops[0]) in [ast.Eq]:
            if self.visit(node.left) is int and\
               self.visit(node.comparators[0]) is int:
                return bool
    
    def visit_Num(self, node):
        if type(node.n) is int:
            return int
    
    def visit_NameConstant(self, node):
        if node.value in [True, False]:
            return bool
    
a = 'pd.DataFrame(data = {1,2})'    

<p>b) DataFrame Cost analysis without runing code